In [9]:
import perceval as pcvl
import perceval.lib.qiskit as lib_qiskit
import perceval.lib.symb as symb

import numpy as np

In [10]:
pcvl.register_plugin("perceval_internal", silent=True)

True

In [11]:
from qiskit import QuantumCircuit

In [12]:
from perceval.converters import qiskit

In [13]:
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.barrier()
circuit.s(0)
circuit.ry(np.pi/4, 1)
circuit.cx(1, 2)
circuit.barrier()
circuit.swap(2, 1)
circuit.rz(np.pi/4, 2)
circuit.cx(0, 2)
circuit.draw()

┌───┐      ░    ┌───┐         ░                    
q_0: ┤ H ├──■───░────┤ S ├─────────░─────────────────■──
     └───┘┌─┴─┐ ░ ┌──┴───┴──┐      ░                 │  
q_1: ─────┤ X ├─░─┤ Ry(π/4) ├──■───░──X──────────────┼──
          └───┘ ░ └─────────┘┌─┴─┐ ░  │ ┌─────────┐┌─┴─┐
q_2: ───────────░────────────┤ X ├─░──X─┤ Rz(π/4) ├┤ X ├
                ░            └───┘ ░    └─────────┘└───┘

In [21]:
pc=qiskit.converter(circuit, heralded=False)
pcvl.pdisplay(pc, recursive=True, render_size=0.5)

In [15]:
u=pc.compute_unitary(use_symbolic=False)

In [16]:
ub = (pcvl.Circuit(2)
      // (0, symb.PS(phi=pcvl.Parameter("φ_a")))
      // symb.BS(theta=pcvl.Parameter("theta")))

pc_norm = pcvl.Circuit.decomposition(u, ub, shape="rectangle")
pcvl.pdisplay(pc_norm, compact=True, render_size=0.8)
